# ДЗ 4 
## W2V, bert, оценка качества

### __Задача__1:

Реализуйте поиск, где
- метод векторизации текстов - **fasttext** (модель araneum_none_fasttextcbow_300_5_2018) и **Bert** (модель sbert_large_nlu_ru)
- формат хранения индекса - **матрица Document-Term**
- метрика близости пар (запрос, документ) - **косинус**, он же dot на нормированных векторах 

В реализации должно быть все то же, что вcегда:
- функция индексации корпуса
- функция индексации запроса
- функция с реализацией подсчета близости запроса и документов корпуса
- главная функция, объединяющая все это вместе; на входе - запрос, на выходе - отсортированные по убыванию несколько (5) текстов коллекции

Сортировку надо сделать **<font color='green'>обязательно векторно</font>** через маску; при не соблюдении минус два балла. Решение должно учитывать  **<font color='green'> комментарии за предыдущие работы</font>**; при не соблюдении минус балл за каждый пункт.


В качестве корпуса возьмите корпус вопросов и ответов с Ответы Мейл 👍
Описание структуры данных можно посмотреть по ссылке. В качестве документов корпуса берем значения из ключа *answers*, но не все, а один, у которого максимальный *value*. Ограничиваем количество документов до 50000. 


**На что направлена эта задача:** 
Реализация поисковика с использованием других векторных моделей, а именно fasttext и Bert.



### __Задача__2:
**Сравните** качество поиска в зависимости от метода векторизации, а именно:
1. CountVectorizer
2. TfidfVectorizer
3. BM25
4. fasttext
5. bert

В реализации должно быть:
- функция, которая считает итоговую метрику на топ-5 результатов по логике, описанной в лекции


👉 В результате нужно вывести значение метрики по каждому методу.

In [1]:
from pymystem3 import Mystem
from string import punctuation
import nltk
from nltk.corpus import stopwords
import re
from scipy import sparse
import json
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from pprint import pprint
import pickle
from gensim.models import KeyedVectors
from tqdm import tqdm
import torch
from transformers import AutoTokenizer, AutoModel
from time import time


m = Mystem()
nltk.download('stopwords')
stopword = stopwords.words('russian')
punctuation += '...' + '—' + '…' + '«»'

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\trekc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [25]:
def first_processing(file):
    with open(file, 'r', encoding='utf-8') as f:
        corpus = list(f)[:10500]

    answers_corpus = []
    dropped = []
    
    for i, part in enumerate(corpus):
        d = dict()
        j_answers = json.loads(part)['answers']

        try: # бывает пустое поле answers
            for ind, gr_comments in enumerate(j_answers):

                try:
                    d[ind] = int(gr_comments['author_rating']['value'])

                except ValueError: # бывает пустое поле value
                    d[ind] = 0

            ind = sorted(d.items(), key=lambda item: item[1], reverse=True)[0][0]
            answers_corpus.append(j_answers[ind]['text'])

        except IndexError:
            dropped.append(i)
            pass
    
    
    questions_corpus = []

    for i, part in enumerate(corpus):
        if i not in dropped:
            questions_corpus.append(json.loads(part)['question'])
    
    
    
    return answers_corpus, questions_corpus

In [26]:
answers_corpus, questions_corpus = first_processing('questions_about_love.jsonl')

In [27]:
len(answers_corpus)

10171

In [28]:
def second_processing(given_corpus):
    corpus = []
    dropped = []
    for text in given_corpus:
        text = re.sub('[0-9a-zA-Z]+', '', text)
        text = [word.lower().strip().strip(punctuation) for word in text.split()]
        #text = [x for x in text if x not in stopword]
        text = ' '.join([word for word in text if word != ''])
        corpus.append(text)
    
    # чтобы удалить пустые строки и сохранить их индексы
    # чтобы убрать их в изначальном датасете
    for ind, text in enumerate(corpus):
        if not text:
            dropped.append(ind)
            del corpus[ind]
        if text == '':
            dropped.append(ind)
            del corpus[ind]        
        
    lol = lambda lst, sz: [lst[i:i+sz] for i in range(0, len(lst), sz)]
    txtpart = lol(corpus, 1000)
    res = []
    for txtp in txtpart:
        alltexts = ' '.join([txt + ' br ' for txt in txtp])
        words = m.lemmatize(alltexts)
        doc = []
        for txt in words:
            if txt != '\n' and txt.strip() != '':
                if txt == 'br':
                    res.append(' '.join(doc))
                    doc = []
                else:
                    doc.append(txt)

    return res, dropped

In [29]:
def get_cleared_corpus(answers_corpus, questions_corpus):
    cleared_answers_corpus, ans_dropped = second_processing(answers_corpus)
    
    for ind in ans_dropped:
        del questions_corpus[ind]
    
    for ind in ans_dropped:
        del answers_corpus[ind]
        
    cleared_questions_corpus, ques_dropped = second_processing(questions_corpus)
    
    for ind in ques_dropped:
        del cleared_answers_corpus[ind]
    
    for ind in ques_dropped:
        del questions_corpus[ind]

    for ind in ques_dropped:
        del answers_corpus[ind]
    
    return answers_corpus, questions_corpus, cleared_answers_corpus, cleared_questions_corpus

In [30]:
answers_corpus, questions_corpus, cleared_answers_corpus, cleared_questions_corpus = get_cleared_corpus(answers_corpus, questions_corpus)

In [32]:
len(answers_corpus)

10095

In [36]:
# with open('answers_corpus.pickle', 'wb') as f:
#     pickle.dump(answers_corpus, f)
    
# with open('questions_corpus.pickle', 'wb') as f:
#     pickle.dump(questions_corpus, f)
    
# with open('cleared_answers_corpus.pickle', 'wb') as f:
#     pickle.dump(cleared_answers_corpus, f)
    
# with open('cleared_questions_corpus.pickle', 'wb') as f:
#     pickle.dump(cleared_questions_corpus, f)

In [34]:
with open('answers_corpus.pickle', 'rb') as f:
    answers_corpus = pickle.load(f)
    
with open('questions_corpus.pickle', 'rb') as f:
    questions_corpus = pickle.load(f)
    
with open('cleared_answers_corpus.pickle', 'rb') as f:
    cleared_answers_corpus = pickle.load(f)
    
with open('cleared_questions_corpus.pickle', 'rb') as f:
    cleared_questions_corpus = pickle.load(f)

In [35]:
len(cleared_questions_corpus)

9595

### Индексация

In [16]:
from gensim.models import FastText
from gensim.models import KeyedVectors
from gensim.test.utils import common_texts

fasttext_model = KeyedVectors.load('araneum_none_fasttextcbow_300_5_2018.model')

In [17]:
def fasttext_get_matrix(texts):
    fasttext_vectors = []
    for i, text in enumerate(texts):
        tokens = text.split()
        tokens_vectors = np.zeros((len(tokens), fasttext_model.vector_size))
        for i, token in enumerate(tokens):
            tokens_vectors[i] = fasttext_model[token]
        if tokens_vectors.shape[0] != 0:
            means = np.mean(tokens_vectors, axis=0)
            n_means = means / np.linalg.norm(means)

        fasttext_vectors.append(n_means)
        
    return sparse.csr_matrix(fasttext_vectors)

In [35]:
w = np.dot(fasttext_ques_matrix, fasttext_ans_matrix.T)

In [40]:
for ind, line in enumerate(w):
    print(w)

  (0, 9594)	0.00961520446305805
  (0, 9593)	0.6523250896695375
  (0, 9592)	-0.13981960271033955
  (0, 9591)	0.4597239541605544
  (0, 9590)	0.6143178112539777
  (0, 9589)	0.03760958256718471
  (0, 9588)	0.32342025915590744
  (0, 9587)	0.2210767609354648
  (0, 9586)	0.32340823608431796
  (0, 9585)	0.2466208661382731
  (0, 9584)	-0.037977391123021724
  (0, 9583)	0.4783842052287916
  (0, 9582)	0.4711405571184212
  (0, 9581)	0.4676759028345956
  (0, 9580)	0.5468900726056373
  (0, 9579)	0.4077364055331764
  (0, 9578)	0.16051731538256986
  (0, 9577)	0.5021259474680448
  (0, 9576)	0.15942950539765396
  (0, 9575)	0.3467151362278232
  (0, 9574)	-0.10333792000843298
  (0, 9573)	0.16081291571106607
  (0, 9572)	0.4957318954832757
  (0, 9571)	0.5596045319624955
  (0, 9570)	-0.09620597406620335
  :	:
  (9594, 24)	-0.08335278662613792
  (9594, 23)	0.14034464419742687
  (9594, 22)	0.03324168959380944
  (9594, 21)	0.25227466990962194
  (9594, 20)	0.020710149434398265
  (9594, 19)	0.24363099288927217
  (

KeyboardInterrupt: 

In [36]:
#fasttext_ans_matrix = fasttext_get_matrix(cleared_answers_corpus)
#fasttext_ques_matrix = fasttext_get_matrix(cleared_questions_corpus)
#sparse.save_npz('fasttext_ans_matrix.npz', fasttext_ans_matrix)
fasttext_ans_matrix = sparse.load_npz('fasttext_ans_matrix.npz')
#sparse.save_npz('fasttext_ques_matrix.npz', fasttext_ques_matrix)
fasttext_ques_matrix = sparse.load_npz('fasttext_ques_matrix.npz')

### Реализация поиска

In [20]:
def get_similarity(sparced_matrix, query_vec):
    scores = cosine_similarity(sparced_matrix, query_vec)
    sorted_scores_indx = np.argsort(scores, axis=0)[::-1]
    return list(np.array(answers_corpus)[sorted_scores_indx.ravel()][:5])

In [21]:
def fasttext_query_preprocessing(query):
    query = [word.lower().strip(punctuation).strip() for word in query.split()]
    query = m.lemmatize(' '.join([word for word in query]))
    query = ' '.join([word for word in query])
    tokens = [word for word in query.split() if word != '']
    query_vectors = []
    tokens_vectors = np.zeros((len(tokens), fasttext_model.vector_size))

    for i, token in enumerate(tokens):
        tokens_vectors[i] = fasttext_model[token]
    if tokens_vectors.shape[0] != 0:
        means = np.mean(tokens_vectors, axis=0)
        n_means = means / np.linalg.norm(means)
        query_vectors.append(n_means)

        return sparse.csr_matrix(query_vectors)

    else:
        return None

In [22]:
def fasttext_search():
    while True:
        query = input('Введите запрос (или "ОСТАНОВИТЕ" для остановки):')
        if query == 'ОСТАНОВИТЕ':
            break
        start = time()
        query_vec = fasttext_query_preprocessing(query)
        if query_vec is None:
            continue
        else:
            pprint(get_similarity(fasttext_ques_matrix, query_vec))
            end = time()
            print('Time spent for search - ', end - start)

In [23]:
def scoring(q_matrix, a_matrix):
    #q_matrix = np.delete(q_matrix.toarray(), np.s_[10000:], 0)
    #a_matrix = np.delete(a_matrix.toarray(), np.s_[10000:], 0)

    scoring_matrix = np.dot(q_matrix, a_matrix.T)
    score = 0
    for ind, line in enumerate(scoring_matrix):

        sorted_scores_indx = np.argsort(line, axis=0)[::-1]
        sorted_scores_indx = [sorted_scores_indx.ravel()][0][:5]
        if ind in sorted_scores_indx:
            score += 1

    return score / q_matrix.shape[0]

In [43]:
#fasttext_ques_matrix.toarray()[:25000]

In [38]:
start = time()
print(scoring(fasttext_ques_matrix[:10000].toarray(), fasttext_ans_matrix[:10000].toarray()))
end = time()
print(end - start)

301
0.031370505471599794
5.77693247795105


In [33]:
fasttext_search()

Введите запрос (или "ОСТАНОВИТЕ" для остановки):sdas


MemoryError: Unable to allocate 680. MiB for an array with shape (48582,) and data type <U3667

### Bert

In [9]:
from tqdm import tqdm
import torch, torchvision
from transformers import AutoTokenizer, AutoModel

In [10]:
auto_tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/sbert_large_nlu_ru")
auto_model = AutoModel.from_pretrained("sberbank-ai/sbert_large_nlu_ru")
auto_model.to('cuda')

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(120138, 1024, padding_idx=0)
    (position_embeddings): Embedding(512, 1024)
    (token_type_embeddings): Embedding(2, 1024)
    (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=1024, out_features=1024, bias=True)
            (key): Linear(in_features=1024, out_features=1024, bias=True)
            (value): Linear(in_features=1024, out_features=1024, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=1024, out_features=1024, bias=True)
            (LayerNorm): LayerNorm((1024,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=Fal

In [11]:
def cls_pooling(model_output, attention_mask):
    return model_output[0][:,0]

def bert_vectorizer(corpus):
    corpus = [corpus[i:i + 3250] for i in range(0, len(corpus), 3250)]
    bert_vects = []
    for text in tqdm(corpus):
        encoded_input = auto_tokenizer(text, padding=True, truncation=True, max_length=24, return_tensors='pt')
        encoded_input = encoded_input.to('cuda')
        with torch.no_grad():
            model_output = auto_model(**encoded_input)
        
        sentence_embeddings = cls_pooling(model_output, encoded_input['attention_mask'])
        sentence_embeddings = torch.nn.functional.normalize(sentence_embeddings)
        for sentence in sentence_embeddings:
            bert_vects.append(sentence.cpu().numpy())
    torch.cuda.empty_cache()
    
    return sparse.csr_matrix(bert_vects)

In [12]:
#torch.save(bert_vectorizer(questions_corpus), 'ques_bert.pt')
#torch.save(bert_vectorizer(answers_corpus), 'ans_bert.pt')
b_answers = torch.load('ans_bert.pt')
b_questions = torch.load('ques_bert.pt')

In [35]:
def bert_search():
    while True:
        query = input('Введите запрос (или "ОСТАНОВИТЕ" для остановки):')
        if query == 'ОСТАНОВИТЕ':
            break
        query_vec = bert_vectorizer(query)
        pprint(get_similarity(b_questions, query_vec))

In [53]:
bert_search()

Введите запрос (или "ОСТАНОВИТЕ" для остановки):почему он не дарит мне цветы?


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  7.88it/s]


['думаю об этом следует поговорить с ним, а не нами',
 'а вот если в постель сразу потащил, писала бы тут, что бабник и извращенец... что тебя не устраивает сейчас? ситара мали',
 'Спой ему песню https://www.youtube.com/watch?v=hme25Nm3JXg',
 'Скажи ему "милый, ну когда?"',
 'Видать ты хороша... во всём.']
Введите запрос (или "ОСТАНОВИТЕ" для остановки):ОСТАНОВИТЕ


In [15]:
start = time.time()
scoring(b_questions, b_answers)
end = time.time()
print(end - start)

0.0226
7.5947370529174805


### BM25


In [4]:
def bm25_query_preprocessing(query, count_vectorizer):
    query = [word.lower().strip(punctuation).strip() for word in query.split()]
    query = m.lemmatize(' '.join([word for word in query]))
    query = ' '.join([word for word in query])
    query = ' '.join([word for word in query.split() if word != ''])
    query_vec = count_vectorizer.transform([query])
    return query_vec

In [5]:
def bm25_vectorization(ans_cleared_corpus, que_cleared_corpus):
    count_vectorizer = CountVectorizer()
    tf_vectorizer = TfidfVectorizer(use_idf=False, norm='l2')
    tfidf_vectorizer = TfidfVectorizer(use_idf=True, norm='l2')

    x_count_vec = count_vectorizer.fit_transform(ans_cleared_corpus)  # для индексации запроса
    x_tf_vec = tf_vectorizer.fit_transform(ans_cleared_corpus)  # матрица с tf
    x_tfidf_vec = tfidf_vectorizer.fit_transform(ans_cleared_corpus)  # матрица для idf
    idf = tfidf_vectorizer.idf_
    idf = np.expand_dims(idf, axis=0)
    tf = x_tf_vec

    values = []
    rows = []
    cols = []
    k = 2
    b = 0.75

    len_d = x_count_vec.sum(axis=1)
    avdl = len_d.mean()
    B_1 = (k * (1 - b + b * len_d / avdl))

    for i, j in zip(*tf.nonzero()):
        rows.append(i)
        cols.append(j)
        A = idf[0][j] * tf[i, j] * k + 1
        B = tf[i, j] + B_1[i]
        AB = A / B

        values.append(np.asarray(AB)[0][0])
    
    bm25_answers = sparse.csr_matrix((values, (rows, cols)))
    bm25_questions = count_vectorizer.transform(que_cleared_corpus)
    return bm25_answers, bm25_questions, count_vectorizer

In [11]:
    with open('bm25_answers.npz', 'rb') as f:
        bm25_answers = pickle.load(f)
    with open('bm25_questions.npz', 'rb') as f:
        bm25_questions = pickle.load(f)
    with open('bm25_count_vectorizer.pickle', 'rb') as f:
        bm25_count_vectorizer = pickle.load(f)

In [7]:
bm25_answers[:10000]

<10000x30882 sparse matrix of type '<class 'numpy.float64'>'
	with 106216 stored elements in Compressed Sparse Row format>

In [25]:
bm25_answers, bm25_questions, bm25_count_vectorizer = bm25_vectorization(cleared_answers_corpus[:10000], cleared_questions_corpus[:10000])

In [258]:
def bm25_tfidf_count_search(vectorizer):
    while True:
        query = input('Введите запрос (или "ОСТАНОВИТЕ" для остановки):')
        if query == 'ОСТАНОВИТЕ':
            break
        query_vec = bm25_query_preprocessing(query, vectorizer)
        pprint(get_similarity(bm25_questions, query_vec))

In [251]:
bm25_tfidf_count_search(bm25_count_vectorizer)

Введите запрос (или "ОСТАНОВИТЕ" для остановки):как бросить курить
['Да, это весьма пагубная привычка)Но будь готов к тому, что это будет не легко...',
 'что бы быть харизматичными, нужно, как ты правильно заметила, быть самодостаточными, а для этого нужно научиться себя любить и наполнить себя до краёв любовью, а вот излишки, которые будут изливаться на окружающих людей- это и есть то самое, что привлекает окружающих',
 'Видно решил с детьми больше дел не иметь.',
 'я бы не за него порадовалась, а за себя, что это все раскрылось довольно быстро есть такие мужчины, которые долгие годы не проявляют свою подлую натуру, а потом бросают жену с маленькими детьми и фотки с мальдивов шлютвполне естественно ненавидеть предателя, а не радоваться за негои кстати, он за вас порадовался бы ой как вряд ли в такой же ситуации, а возможно еще и гораздо хуже реакция была быэтот человек прежде всего эгоист, я предполагаю, и пока его не задевают, он о чувствах другого человека очень мало задумываетсянав

In [15]:
# принимает спарс матрицы
def scoring_for_count(q_matrix, a_matrix):
    # q_matrix = np.delete(q_matrix., np.s_[5000:], 0)
    # a_matrix = np.delete(a_matrix.toarray(), np.s_[5000:], 0)
    scoring_matrix = np.dot(q_matrix[:10000].toarray(), a_matrix[:10000].toarray().T)
    score = 0
    for ind, line in enumerate(scoring_matrix):
        sorted_scores_indx = np.argsort(line, axis=0)[::-1]
        sorted_scores_indx = [sorted_scores_indx.ravel()][0][:5]
        if ind in sorted_scores_indx:
            score += 1
    
    print(score/10000)

In [14]:
scoring_for_count(bm25_questions, bm25_answers)

0.048


In [24]:
bm25_questions[:10000].toarray()[2].sum()

4

### TF IDF

In [245]:
def tfidf_vectorization(ans_cleared_corpus, que_cleared_corpus):
    vectorizer = TfidfVectorizer()
    tfidf_answers = vectorizer.fit_transform(ans_cleared_corpus)
    tfidf_questions = vectorizer.transform(que_cleared_corpus)
    
    return tfidf_answers, tfidf_questions, vectorizer

In [255]:
tfidf_answers, tfidf_questions, tfidf_vectorizer = tfidf_vectorization(cleared_answers_corpus[:10000], cleared_questions_corpus[:10000])

In [260]:
bm25_tfidf_count_search(tfidf_count_vectorizer)

Введите запрос (или "ОСТАНОВИТЕ" для остановки):почему он не дарит мне цветы?
['думаю об этом следует поговорить с ним, а не нами',
 'Потому что посмотришь на этих коров, и кажется, что они твой букет сожрут.)',
 'Вика! Я - цветы любимой на КАЖДЫЙ день рождения!Всего 42 раза!Надеюсь!Подарить еще раз двадцать.В остальное время - шоколад и шоколадные конфетки.Она их обожает!',
 'Любому артисту приятно получить цветы именно на сцене. Так что дарите, не стесняйтесь!',
 'Заварку']
Введите запрос (или "ОСТАНОВИТЕ" для остановки):ОСТАНОВИТЕ


In [265]:
scoring_for_count(tfidf_questions, tfidf_answers)

0.0783


### Count

In [49]:
def count_vectorization(ans_cleared_corpus, que_cleared_corpus):
    vectorizer = TfidfVectorizer(use_idf=False, norm='l2')
    count_answers = vectorizer.fit_transform(ans_cleared_corpus)
    count_questions = vectorizer.transform(que_cleared_corpus)
    
    return count_answers, count_questions, vectorizer

In [50]:
count_answers, count_questions, count_vectorizer = count_vectorization(cleared_answers_corpus[:10000], cleared_questions_corpus[:10000])

In [259]:
bm25_tfidf_count_search(count_vectorizer)

Введите запрос (или "ОСТАНОВИТЕ" для остановки):как бросить курить?
['Да, это весьма пагубная привычка)Но будь готов к тому, что это будет не легко...',
 'что бы быть харизматичными, нужно, как ты правильно заметила, быть самодостаточными, а для этого нужно научиться себя любить и наполнить себя до краёв любовью, а вот излишки, которые будут изливаться на окружающих людей- это и есть то самое, что привлекает окружающих',
 'Видно решил с детьми больше дел не иметь.',
 'я бы не за него порадовалась, а за себя, что это все раскрылось довольно быстро есть такие мужчины, которые долгие годы не проявляют свою подлую натуру, а потом бросают жену с маленькими детьми и фотки с мальдивов шлютвполне естественно ненавидеть предателя, а не радоваться за негои кстати, он за вас порадовался бы ой как вряд ли в такой же ситуации, а возможно еще и гораздо хуже реакция была быэтот человек прежде всего эгоист, я предполагаю, и пока его не задевают, он о чувствах другого человека очень мало задумываетсяна

In [264]:
scoring_for_count(count_questions, count_answers)

KeyboardInterrupt: 